In [3]:
FUENTES_DIR = '../Fuentes'         # carpeta donde se encuentran archivos .py auxiliares
DATOS_DIR   = '../Data_Sets/p3/' # carpeta donde se encuentran los datasets

# agrega ruta de busqueda donde tenemos archivos .py
import sys
sys.path.append(FUENTES_DIR)

%matplotlib inline
import numpy as np
import pandas as pd
from matplotlib import pylab as plt
from IPython import display
from ClassPerceptron import Perceptron
from sklearn import preprocessing
import grafica as gr
from ClassNeuronaGral import *
from sklearn.model_selection import train_test_split

## Ejercicio 9

Se ha realizado un análisis químico a tres tipos distintos de vinos producidos en una misma región de Italia. El número de muestras considerado es el siguiente:

$$
	\text{Tipo 1} \rightarrow 59~\text{muestras}, \quad\quad \text{Tipo 2} \rightarrow 71~\text{muestras}, \quad\quad \text{Tipo 3} \rightarrow 48~\text{muestras}
$$


El archivo **vinos.csv** permite observar los resultados de este análisis. Cada fila representa una muestra distinta y está formada, en primer lugar, por el número del tipo al cual pertenece el vino analizado, seguido por los 13 atributos que lo caracterizan.  

Por ejemplo, la siguiente fila:

$$
	2, ~12.29, ~3.17, ~2.21, ~18, ~88, ~2.85, ~2.99, ~0.45, ~2.81, ~2.3, ~1.42, ~2.83, ~406
$$


es el resultado del análisis de un vino correspondiente al **tipo 2** (primer valor de la fila), seguido por 13 valores separados por comas que indican los niveles de las mediciones realizadas a dicho vino.


### a) 

Entrene una red neuronal formada por una única neurona para clasificar los vinos de **Tipo 1**.

Realice **30 ejecuciones independientes** utilizando el **50%, 60%, 70%, 80% y 90%** de los ejemplos como entrenamiento y el resto como testeo. Para cada porcentaje, indique la cantidad promedio de ejemplos correctamente clasificados en entrenamiento y en testeo. Calcule también el promedio y el desvío de la cantidad de iteraciones realizadas.

Utilice un máximo de 400 iteraciones y velocidades de aprendizaje 0.1, 0.2 y 0.3.

Analice los resultados obtenidos utilizando:
**i.** Función de activación ‘sigmoid’ y función de costo ‘ECM’ (error cuadrático medio)
**ii.** Función de activación ‘sigmoid’ y función de costo ‘EC_binaria’ (entropía cruzada binaria)
**iii.** Función de activación ‘tanh’ y función de costo ‘ECM’ (error cuadrático medio)



# *i* & *ii*

In [11]:
data = pd.read_csv(DATOS_DIR + 'Vinos.csv', sep= ',')
ejecuciones = 30
porcentajes = [0.5, 0.6, 0.7, 0.8, 0.9]
alphas = [0.1, 0.2, 0.3]
N_ITER = 400
COTA = 1e-4
# Separar atributos y clase
X = data.iloc[:, 1:].values          # 13 atributos
y = (data.iloc[:, 0].values == 1).astype(int)  # 1 si es tipo 1, 0 en caso contrario

# -------------------
# Estructura para guardar resultados
# -------------------
resultados = {}

for alpha in alphas:
    resultados[alpha] = {}
    for p in porcentajes:
        resultados[alpha][p] = []
        for rep in range(ejecuciones): 
            # split train / test
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, train_size=p, shuffle=True
            )

            # entreno la neurona
            ng = ng = NeuronaGradiente(alpha=alpha, n_iter=N_ITER, cotaE=COTA, FUN='sigmoid')
            ng.fit(X_train, y_train)

            # predicciones
            y_pred = ng.predict(X_test)        # etiquetas binarias
            y_prob = ng.predict_nOut(X_test)   # salida sigmoidal

            # métricas
            accuracy = np.mean(y_pred == y_test)
            ecm = np.mean((y_prob - y_test) ** 2)
            eps = np.finfo(float).eps  # evitar log(0)
            cross_entropy = -np.mean(y_test * np.log(y_prob + eps) + (1 - y_test) * np.log(1 - y_prob + eps))
            resultados[alpha][p].append((accuracy, ecm, cross_entropy)) 

# impresión de resultados
for alpha in alphas:
    print(f"Resultados para alpha = {alpha}:")
    for p in porcentajes:
        acc_list  = [acc for acc, _, _ in resultados[alpha][p]]
        ecm_list  = [ecm for _, ecm, _ in resultados[alpha][p]]
        ce_list   = [ce  for _, _, ce in resultados[alpha][p]]

        print(f"  Porcentaje entrenamiento {int(p*100)}%:")
        print(f"    Accuracy -> Media = {np.mean(acc_list):.4f}, Desvío = {np.std(acc_list):.4f}")
        print(f"    ECM      -> Media = {np.mean(ecm_list):.6f}, Desvío = {np.std(ecm_list):.6f}")
        print(f"    CE bin   -> Media = {np.mean(ce_list):.6f}, Desvío = {np.std(ce_list):.6f}")


Resultados para alpha = 0.1:
  Porcentaje entrenamiento 50%:
    Accuracy -> Media = 1.0000, Desvío = 0.0000
    ECM      -> Media = 0.000254, Desvío = 0.000000
    CE bin   -> Media = 0.016055, Desvío = 0.000011
  Porcentaje entrenamiento 60%:
    Accuracy -> Media = 1.0000, Desvío = 0.0000
    ECM      -> Media = 0.000213, Desvío = 0.000000
    CE bin   -> Media = 0.014688, Desvío = 0.000011
  Porcentaje entrenamiento 70%:
    Accuracy -> Media = 1.0000, Desvío = 0.0000
    ECM      -> Media = 0.000181, Desvío = 0.000000
    CE bin   -> Media = 0.013563, Desvío = 0.000009
  Porcentaje entrenamiento 80%:
    Accuracy -> Media = 1.0000, Desvío = 0.0000
    ECM      -> Media = 0.000158, Desvío = 0.000000
    CE bin   -> Media = 0.012664, Desvío = 0.000005
  Porcentaje entrenamiento 90%:
    Accuracy -> Media = 1.0000, Desvío = 0.0000
    ECM      -> Media = 0.000141, Desvío = 0.000000
    CE bin   -> Media = 0.011924, Desvío = 0.000010
Resultados para alpha = 0.2:
  Porcentaje entrenami

# *iii*

In [15]:
y = np.where(data.iloc[:,0].values == 1, 1, -1)


resultados = {}

for alpha in alphas:
    resultados[alpha] = {}
    for p in porcentajes:
        resultados[alpha][p] = []
        for rep in range(ejecuciones): 
            # split train / test
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, train_size=p, shuffle=True
            )

            # entreno la neurona
            ng = ng = NeuronaGradiente(alpha=alpha, n_iter=N_ITER, cotaE=COTA, FUN='tanh')
            ng.fit(X_train, y_train)

            # predicciones
            y_pred = ng.predict(X_test)        # etiquetas binarias
            y_prob = ng.predict_nOut(X_test)   # salida sigmoidal

            # métricas
            accuracy = np.mean(y_pred == y_test)
            ecm = np.mean((y_prob - y_test) ** 2)
            resultados[alpha][p].append((accuracy, ecm)) 

# impresión de resultados
for alpha in alphas:
    print(f"Resultados para alpha = {alpha}:")
    for p in porcentajes:
        acc_list  = [acc for acc, _ in resultados[alpha][p]]
        ecm_list  = [ecm for _, ecm in resultados[alpha][p]]

        print(f"  Porcentaje entrenamiento {int(p*100)}%:")
        print(f"    Accuracy -> Media = {np.mean(acc_list):.4f}, Desvío = {np.std(acc_list):.4f}")
        print(f"    ECM      -> Media = {np.mean(ecm_list):.6f}, Desvío = {np.std(ecm_list):.6f}")


Resultados para alpha = 0.1:
  Porcentaje entrenamiento 50%:
    Accuracy -> Media = 1.0000, Desvío = 0.0000
    ECM      -> Media = 0.000125, Desvío = 0.000000
  Porcentaje entrenamiento 60%:
    Accuracy -> Media = 1.0000, Desvío = 0.0000
    ECM      -> Media = 0.000105, Desvío = 0.000000
  Porcentaje entrenamiento 70%:
    Accuracy -> Media = 1.0000, Desvío = 0.0000
    ECM      -> Media = 0.000089, Desvío = 0.000000
  Porcentaje entrenamiento 80%:
    Accuracy -> Media = 1.0000, Desvío = 0.0000
    ECM      -> Media = 0.000078, Desvío = 0.000000
  Porcentaje entrenamiento 90%:
    Accuracy -> Media = 1.0000, Desvío = 0.0000
    ECM      -> Media = 0.000069, Desvío = 0.000000
Resultados para alpha = 0.2:
  Porcentaje entrenamiento 50%:
    Accuracy -> Media = 1.0000, Desvío = 0.0000
    ECM      -> Media = 0.000087, Desvío = 0.000000
  Porcentaje entrenamiento 60%:
    Accuracy -> Media = 1.0000, Desvío = 0.0000
    ECM      -> Media = 0.000073, Desvío = 0.000000
  Porcentaje entre